# Simple Offline Single-Agent vs Multi-Agent (LangGraph + Ollama + Local Embeddings)

This lesson demonstrates two ways to orchestrate LLM work:

- **Single-agent**: one agent does everything end-to-end.
- **Multi-agent**: multiple specialized agents collaborate (plan → retrieve → write).

Everything runs **locally**:
- **Ollama** runs the LLM on your machine (no cloud keys).
- **sentence-transformers** generates embeddings locally.
- **FAISS** performs local vector similarity search.

**Sources**
- [FAISS](https://faiss.ai/index.html)
- [LangGraph/LangChain](https://www.langchain.com/langgraph)

In [29]:
%pip -q install -U langgraph langchain langchain-community langchain-text-splitters sentence-transformers faiss-cpu pandas langchain-ollama


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [30]:
import pandas as pd
from typing import TypedDict, List
from pathlib import Path
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END

## Step 1 — Load the dataset (CSV → Documents)

[Data source](https://www.kaggle.com/datasets/sumitm004/arxiv-scientific-research-papers-dataset?resource=download)

In [31]:
CSV_PATH = Path("data/arXiv_scientific_dataset.csv")
print("Loading:", CSV_PATH)

df = pd.read_csv(CSV_PATH)
print("Rows:", len(df), "| Cols:", len(df.columns))
print("Columns:", list(df.columns))

TEXT_COL = "summary"
print("Using text column:", TEXT_COL)

if TEXT_COL not in df.columns:
    raise ValueError(f"Missing '{TEXT_COL}' column")

print("Missing summaries:", df[TEXT_COL].isna().sum())


def row_to_text(r):
    title = (
        str(r["title"])
        if "title" in df.columns and pd.notna(r.get("title", None))
        else ""
    )
    body = str(r[TEXT_COL]) if pd.notna(r[TEXT_COL]) else ""
    return f"Title: {title}\nAbstract: {body}" if title else body


docs = [
    Document(page_content=row_to_text(r), metadata={"row": int(i)})
    for i, r in df.iterrows()
]
docs = [d for d in docs if d.page_content.strip()]
print("Docs:", len(docs))
print("\nExample doc:\n", docs[0].page_content[:500])

Loading: data/arXiv_scientific_dataset.csv
Rows: 3000 | Cols: 10
Columns: ['id', 'title', 'category', 'category_code', 'published_date', 'updated_date', 'authors', 'first_author', 'summary', 'summary_word_count']
Using text column: summary
Missing summaries: 0
Docs: 3000

Example doc:
 Title: Dynamic Backtracking
Abstract: Because of their occasional need to return to shallow points in a search
tree, existing backtracking methods can sometimes erase meaningful progress
toward solving a search problem. In this paper, we present a method by which
backtrack points can be moved deeper in the search space, thereby avoiding this
difficulty. The technique developed is a variant of dependency-directed
backtracking that uses only polynomial space while still providing useful
control in


## Step 2 — Build a local retriever (Chunk → Embed → Index)

A “retriever” finds the most relevant text passages for a question.

We build it in three small steps:

1. **Chunking**  
   Split long text into overlapping passages. This improves retrieval accuracy and avoids losing context at boundaries.

2. **Embeddings (local)**  
   Convert each chunk into a vector using a local sentence-transformers model.

3. **Vector index (FAISS)**  
   Store vectors so we can efficiently run similarity search.

Outputs of this step:
- `retrieve(query, k)` returns top‑k relevant chunks
- `evidence_block(...)` formats those chunks for the prompt

In [ ]:
print("Chunking documents...")
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=80)
chunks = splitter.split_documents(docs)
print("Chunks:", len(chunks))
print("Example chunk:\n", chunks[0].page_content[:300])

print("\nLoading embedding model...")
emb = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "mps"}
)
print("Embedding model ready.")

print("\nBuilding FAISS index...")
vs = FAISS.from_documents(chunks, emb)
print("FAISS index ready.")


def retrieve(query: str, k: int = 5) -> List[Document]:
    print(f"\n[retrieve] Query: {query} | top_k={k}")
    results = vs.similarity_search(query, k=k)
    print("[retrieve] Results:", len(results))
    return results


def evidence_block(docs: List[Document]) -> str:
    out = []
    for d in docs:
        rid = d.metadata.get("row", "?")
        txt = d.page_content.strip().replace("\n", " ")
        out.append(f"[row={rid}] {txt[:900]}{' ...' if len(txt)>900 else ''}")
    return "\n\n".join(out) if out else "(no matches)"

Chunking documents...
Chunks: 3040
Example chunk:
 Title: Dynamic Backtracking
Abstract: Because of their occasional need to return to shallow points in a search
tree, existing backtracking methods can sometimes erase meaningful progress
toward solving a search problem. In this paper, we present a method by which
backtrack points can be moved deeper

Loading embedding model...


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1729.08it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Embedding model ready.

Building FAISS index...
FAISS index ready.


## Step 3 — Start a local LLM with Ollama

We connect to a locally running Ollama model.

This gives us a chat-style LLM we can call with:
- a **system message** (rules like “use only evidence”)
- a **user message** (question + retrieved evidence)

[Ollama Quickstart Guide](https://docs.ollama.com/quickstart)

In [33]:
llm = ChatOllama(model="ministral-3:8b", temperature=0.2)

# Demo A — Single-Agent Workflow (Retrieve → Answer)

**What it is:**  
A single node/agent does the entire job.

**What happens:**
1. Receive the user question.
2. Retrieve relevant chunks from FAISS.
3. Write the final answer using only those chunks.
4. Cite sources as `[row=...]`.

In [35]:
q = "What are the top 3 open problems in these papers"

In [34]:
class S(TypedDict, total=False):
    question: str
    final: str


def single_node(state: S) -> S:
    q = state["question"]
    ev = evidence_block(retrieve(q, k=5))
    resp = llm.invoke(
        [
            SystemMessage(
                content="Use ONLY the EVIDENCE. Cite as [row=...]. If evidence is weak, say so."
            ),
            HumanMessage(
                content=f"QUESTION:\n{q}\n\nEVIDENCE:\n{ev}\n\nWrite a clear answer."
            ),
        ]
    )
    return {**state, "final": resp.content}


g1 = StateGraph(S)
g1.add_node("single", single_node)
g1.set_entry_point("single")
g1.add_edge("single", END)
single_app = g1.compile()

In [36]:
single_out = single_app.invoke({"question": q})
print(single_out["final"])


[retrieve] Query: What are the top 3 open problems in these papers | top_k=5
[retrieve] Results: 5
Based **solely** on the provided evidence, here are the **top 3 open problems** identified or implied in the papers:

1. **Automating Proofs for Heuristic Properties in Planning**
   - **[row=2228]** The paper discusses that while the heuristic **h+** in classical planning shows remarkable properties (e.g., absence of local minima), these proofs are currently **hand-made**. The open question is whether **domain analysis techniques** can **automatically** derive such proofs, especially given earlier failures in scalable analysis methods.

2. **Generalizing Difficulty Metrics for Complex Problems**
   - **[row=2310]** While Sudoku difficulty metrics are evaluated, the paper suggests that **generalizing these metrics to other problems** (e.g., beyond Sudoku) remains an open challenge. Specifically, the **two sources of difficulty** (individual step complexity and dependency structure) must 

# Demo B — Multi-Agent Workflow (Planner → Librarian → Writer → Reviewer → Finalizer)

**What it is:**  
A lightweight collaboration pattern where each agent has a single responsibility, and a reviewer enforces grounding and clarity.

**Agents (nodes):**
1. **Planner**
   - Produces a short plan
   - Proposes several retrieval queries (different angles on the question)

2. **Librarian**
   - Executes retrieval for each query against the FAISS index
   - Consolidates the best chunks into one **evidence package** for the rest of the team

3. **Writer**
   - Writes a **draft** answer
   - Uses only the consolidated evidence
   - Adds citations like `[row=...]` and notes limitations if evidence is weak

4. **Reviewer**
   - Critiques the draft against the evidence
   - Flags unsupported claims, missing citations, and missing trade-offs
   - Outputs required edits (bullet list)

5. **Finalizer**
   - Applies the required edits to produce the final response
   - Keeps the answer grounded in evidence and preserves citations

In [37]:
class M(TypedDict, total=False):
    question: str
    plan: str
    evidence: str
    draft: str
    review: str
    final: str


def planner(state: M) -> M:
    q = state["question"]
    resp = llm.invoke(
        [
            SystemMessage(
                content="Make a short plan and 5 short retrieval queries. Output as bullets."
            ),
            HumanMessage(content=q),
        ]
    )
    return {**state, "plan": resp.content}


def librarian(state: M) -> M:
    lines = [
        ln.strip("-•* ").strip()
        for ln in state["plan"].splitlines()
        if ln.strip().startswith(("-", "•", "*"))
    ]
    queries = lines[:3] if lines else [state["question"]]

    retrieved = []
    for qu in queries:
        retrieved += retrieve(qu, k=3)

    ev = evidence_block(retrieved[:8])
    return {**state, "evidence": ev}


def writer(state: M) -> M:
    resp = llm.invoke(
        [
            SystemMessage(
                content="Use ONLY the EVIDENCE. Cite as [row=...]. Provide trade-offs and limitations."
            ),
            HumanMessage(
                content=(
                    f"QUESTION:\n{state['question']}\n\n"
                    f"PLAN:\n{state['plan']}\n\n"
                    f"EVIDENCE:\n{state['evidence']}\n\n"
                    "Write a DRAFT answer."
                )
            ),
        ]
    )
    return {**state, "draft": resp.content}


def reviewer(state: M) -> M:
    resp = llm.invoke(
        [
            SystemMessage(
                content=(
                    "You are a strict reviewer. Check the DRAFT against the EVIDENCE.\n"
                    "Flag: unsupported claims, missing citations, weak structure, missing trade-offs.\n"
                    "Return:\n"
                    "1) REVIEW (short)\n"
                    "2) REQUIRED_EDITS (bullets)\n"
                )
            ),
            HumanMessage(
                content=(
                    f"QUESTION:\n{state['question']}\n\n"
                    f"EVIDENCE:\n{state['evidence']}\n\n"
                    f"DRAFT:\n{state['draft']}\n"
                )
            ),
        ]
    )
    return {**state, "review": resp.content}


def finalizer(state: M) -> M:
    resp = llm.invoke(
        [
            SystemMessage(
                content="Revise the DRAFT using the REQUIRED_EDITS. Stay grounded in the EVIDENCE and keep citations [row=...]."
            ),
            HumanMessage(
                content=(
                    f"EVIDENCE:\n{state['evidence']}\n\n"
                    f"DRAFT:\n{state['draft']}\n\n"
                    f"REVIEW:\n{state['review']}\n\n"
                    "Return the FINAL answer."
                )
            ),
        ]
    )
    return {**state, "final": resp.content}


g2 = StateGraph(M)
g2.add_node("planner", planner)
g2.add_node("librarian", librarian)
g2.add_node("writer", writer)
g2.add_node("reviewer", reviewer)
g2.add_node("finalizer", finalizer)

g2.set_entry_point("planner")
g2.add_edge("planner", "librarian")
g2.add_edge("librarian", "writer")
g2.add_edge("writer", "reviewer")
g2.add_edge("reviewer", "finalizer")
g2.add_edge("finalizer", END)

multi_app = g2.compile()

In [38]:
multi_out = multi_app.invoke({"question": q})
print(multi_out["final"])


[retrieve] Query:  | top_k=3
[retrieve] Results: 3

[retrieve] Query: Theoretical depth (e.g., P vs. NP variants). | top_k=3
[retrieve] Results: 3

[retrieve] Query: Practical relevance (e.g., scalability, real-world applications). | top_k=3
[retrieve] Results: 3
Here is the revised **DRAFT** incorporating the **REQUIRED_EDITS**, grounded in the **EVIDENCE**, and addressing trade-offs and limitations explicitly:

---

### **Top 3 Open Problems in AI/CS Research**
*(Ranked by theoretical/practical significance, with cross-referenced evidence and trade-offs)*

---

### **1. Scalability of Probabilistic Planning in Uncertain Domains**
**Evidence Base**:
- **[row=1470]**: *"Many problems of interest are complete for PP and NP^PP... deserving of additional study."*
- **[row=1965]**: Sudoku-like CSPs (NP-complete) serve as analogs for probabilistic planning complexity.

**Open Problem**:
While probabilistic planning (e.g., finding small plans in noisy environments) is **PP-hard** (a probabi